In [2]:
import pandas as pd
import sagemaker as sg
from sklearn.model_selection import train_test_split
import boto3 #library used to connect s3bucktes it self

sm_boto3=boto3.client("sagemaker")#client for communicate with s3 bucket
sess=sg.Session()#sagemaker session creation
region=sess.boto_session.region_name#takes default us-east-1
bucket="s3bucketforsagemakerproject"#your s3 bucket name
print("using bucket"+bucket)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\91954\AppData\Local\sagemaker\sagemaker\config.yaml
using buckets3bucketforsagemakerproject


In [3]:
df=pd.read_csv("train.csv")
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [4]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
df.shape

(2000, 21)

In [6]:
#['lowrisk","highrisk"],[0,1]
df['price_range'].value_counts(normalize=True)#classification problem

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [7]:
df.columns#features

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [8]:
df.isnull().mean()*100#percentage of missing values

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [9]:
features=list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [10]:
labels=features.pop(-1)#which is going to predict
labels

'price_range'

In [11]:
x=df[features]
y=df[labels]

In [12]:
x.head()


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [13]:
#{0:'lowrisk',1:'highrisk'}
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [14]:
x.shape

(2000, 20)

In [15]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [16]:
#we can see that data is already balanced as labels has all categories of 500 values

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=0)

In [18]:
print("x_training",x_train.shape)
print("x_test",x_test.shape)
print("y_training",y_train.shape)
print("y_testing",y_test.shape)

x_training (1700, 20)
x_test (300, 20)
y_training (1700,)
y_testing (300,)


In [19]:
trainx=pd.DataFrame(x_train)
trainx[labels]=y_train #basically putting y_train daat over there
testx=pd.DataFrame(x_test)
testx[labels]=y_test#basically putting y_test daat over there
print(trainx)
print(testx)

      battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
1452           1450     0          2.1         0   1       0          31   
1044           1218     1          2.8         1   3       0          39   
1279           1602     0          0.6         0  12       0          58   
674            1034     0          2.6         1   2       1          45   
1200            530     0          2.4         0   1       0          32   
...             ...   ...          ...       ...  ..     ...         ...   
835            1224     1          1.6         0   9       0          33   
1216           1158     0          0.7         1   1       1          29   
1653           1190     0          2.0         1   0       0          40   
559            1191     0          2.4         1   2       0          13   
684             706     0          0.5         0  13       1          31   

      m_dep  mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  \
1452    

In [20]:
print(trainx.shape)
print(trainx.shape)

(1700, 21)
(1700, 21)


In [21]:
#now has features it has 21

In [22]:
trainx.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,...,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,...,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,...,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,...,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,...,48,1012,959,17,7,6,0,1,0,0


In [23]:
trainx.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [24]:
testx.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [25]:
#all above these are feature engineering

In [26]:
#creating separate files for both to upload on s3
trainx.to_csv("train-v-1.csv",index=False)
testx.to_csv("test-v-1.csv",index=False)

In [27]:
bucket#bucket i am using is ....

's3bucketforsagemakerproject'

In [28]:
#sending data to s3.sagemaker will take trainning data from s3
sk_prefix="sagemaker/mobile_price_classification/sklearncontainer"
#creating a folder inside the s3 bucket('s3bucketforsagemakerproject) that is sk_prefix and will upload all th e train,test files to that folder sk_prefix
trainpath=sess.upload_data( #sess is session of library sazemaker
    path="train-v-1.csv",bucket=bucket,key_prefix=sk_prefix
)#upload using session 
testpath=sess.upload_data(
    path="test-v-1.csv",bucket=bucket,key_prefix=sk_prefix
)
print(trainpath)
print(testpath)
#check on your s3 bucket (done) 

s3://s3bucketforsagemakerproject/sagemaker/mobile_price_classification/sklearncontainer/train-v-1.csv
s3://s3bucketforsagemakerproject/sagemaker/mobile_price_classification/sklearncontainer/test-v-1.csv


**TILL NOW WE HAVE TAKEN DATA, DONE TRAIN,TEST SPLIT UPLOADED ON S3 BUCKET I.I. DATA INGESTION PART SO WE HAVE MADE SEESSION ,MANAGED IT,UPLOADING DATA**


In [29]:
%%writefile script.py 
#create script.py used by sagemaker


import argparse
import os
import json
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score
import joblib
import pathlib
from io import StringIO
import boto3
import pandas as pd
import numpy as np

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))#loading model
    return clf

if __name__ =='__main__':#execution starts from here main function
#in sagemaker when you train your model it requires some by default arguments provided by aws sagemaker
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.both below are from randomforestclassification
    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--random_state', type=int, default=0)
    # parser.add_argument('--epochs', type=int, default=10)
    # parser.add_argument('--batch-size', type=int, default=100)
    # parser.add_argument('--learning-rate', type=float, default=0.1)

    # an alternative way to load hyperparameters via SM_HPS environment variable.
    # parser.add_argument('--sm-hps', type=json.loads, default=os.environ['SM_HPS'])

    # input data and model directories and setup in environmments of aws
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
    parser.add_argument('--train_file', type=str, default='train-v-1.csv')
    parser.add_argument('--test_file', type=str, default='test-v-1.csv')

    args, _ = parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))#reading train file from aws
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))#reading test file from aws

    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training RandomForest Model.....")#applying algorithm randomforestclassifier
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=3, n_jobs=None)
    model.fit(X_train, y_train)
    print()
   #dumping model to aws directory
    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Overwriting script.py


In [30]:
#this is also default setup to do from aws documentation
from sagemaker.sklearn.estimator import SKLearn 
#sagemaker has default library SKLearn uses script.py

FRAMEWORK_VERSION = "0.23-1"
#main change is give ARN from IAM as role 
#instance count how many instance to open in sklearn.estimator of sagemaker(basically creates an instance there)
sklearn_estimator = SKLearn(
    entry_point = "script.py",
    role="arn:aws:iam::992382745853:role/amazonsazemakerexecutejob_create_instance",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)
print("created")
#for the 'role' first create a role for allow aws services to makes changes select sagemaker service ,then copy role id from role section paste over here

created


In [32]:
#launch training job,with asynchronous call 
sklearn_estimator.fit({"train":trainpath, "test":testpath}, wait=True)
#just fitting/training the model on test,train values on awstrainpath,testpath finded from session

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-04-20-05-19-35-800


2024-04-20 05:19:39 Starting - Starting the training job...
2024-04-20 05:19:55 Starting - Preparing the instances for training...
2024-04-20 05:20:53 Downloading - Downloading the training image...
2024-04-20 05:21:33 Training - Training image download completed. Training in progress..2024-04-20 05:21:37,414 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-04-20 05:21:37,418 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-20 05:21:37,461 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-04-20 05:21:37,674 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-20 05:21:37,685 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-20 05:21:37,697 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-20 05:21:37,706 sagemaker-training-toolkit INFO     Invoking user 

In [49]:
#deploying training jobs
# predictor = sklearn_estimator.deploy(instance_type='ml.m5.large',
                                     #initial_instance_count=1)

In [36]:
#some more info about model location at( s3 bucket sagemaker etc)
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact present at: ", artifact)


2024-04-20 05:21:59 Starting - Preparing the instances for training
2024-04-20 05:21:59 Downloading - Downloading the training image
2024-04-20 05:21:59 Training - Training image download completed. Training in progress.
2024-04-20 05:21:59 Uploading - Uploading generated training model
2024-04-20 05:21:59 Completed - Training job completed
Model artifact present at:  s3://sagemaker-us-east-1-992382745853/RF-custom-sklearn-2024-04-20-05-19-35-800/output/model.tar.gz


In [37]:
artifact#entire path of model where exist   

's3://sagemaker-us-east-1-992382745853/RF-custom-sklearn-2024-04-20-05-19-35-800/output/model.tar.gz'

In [38]:
#cerating another flder location for deploying entire model
# Making a copy of the built model for deployment
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::992382745853:role/service-role/AmazonSageMaker-ExecutionRole-20240419T230794",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION
)
#role:-amazon sazemaker execution role

In [39]:
model


In [40]:
# Endpoint Deployment
endpoint_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)

EndpointName=Custom-sklearn-model2024-04-20-06-02-05


INFO:sagemaker:Creating model with name: Custom-sklearn-model2024-04-20-06-00-41
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model2024-04-20-06-02-05
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model2024-04-20-06-02-05


-----!

In [41]:
endpoint_name


'Custom-sklearn-model2024-04-20-06-02-05'

In [50]:
predictor

In [43]:
# First five rows of the test dataset for prediction using the endpoint created above
testx[features][0:5].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0],
 [1524.0,
  1.0,
  1.8,
  1.0,
  0.0,
  0.0,
  10.0,
  0.6,
  174.0,
  4.0,
  1.0,
  154.0,
  550.0,
  2678.0,
  16.0,
  5.0,
  13.0,
  1.0,
  0.0,
  1.0],
 [1807.0,
  1.0,
  2.1,
  0.0,
  2.0,
  0.0,
  49.0,
  0.8,
  125.0,
  1.0,
  10.0,
  337.0,
  1384.0,
  1906.0,
  17.0,
  13.0,
  13.0,
  0.0,
  1.0,
  1.0],
 [1086.0,
  1.0,
  1.7,
  1.0,
  0.0,
  1.0,
  43.0,
  0.2,
  111.0,
  6.0,
  1.0,
  56.0,
  1150.0,
  3285.0,
  11.0,
  5.0,
  17.0,
  1.0,
  1.0,
  0.0]]

In [44]:
#printing prediction output for all five data inputed as list
print(predictor.predict(testx[features][0:5].values.tolist()))

[3 0 2 1 3]


In [45]:
# Deleting the endpoint to avoid the costs
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c8132670-58b3-483b-83e6-5b469896639c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c8132670-58b3-483b-83e6-5b469896639c',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 20 Apr 2024 06:09:42 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [46]:
# Deleting the 2ndendpoint to avoid the costs(because i have made 2endpoints)
sm_boto3.delete_endpoint(EndpointName="RF-custom-sklearn-2024-04-20-05-37-32-008")

{'ResponseMetadata': {'RequestId': '49048182-ad77-4fab-b132-3c61e970838f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '49048182-ad77-4fab-b132-3c61e970838f',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 20 Apr 2024 06:11:55 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

**IN documnetation put screenshot of endpoint created**
